**Universidad Politécnica Salesiana**

**Carrera de Ciencias de la Computación**

**Proyecto Integrador Visión por Computadora y Machine Learning**

**Integrantes:** Christian Naranjo, Sara Quiguango, Zelena Jiménez, Leandro Lara

**Nivel:** 9

**Fecha:** 31/07/2024

**Código para segmentación, SIFT, MATCH y generación de csv final**

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
from sklearn.cluster import KMeans

# Definir las carpetas
permitidos_folder = 'Alquerque/permitidos'
no_permitidos_folder = 'Alquerque/no_permitidos'

# Preparar imágenes de referencia para círculos y cruces
ref_circle = cv2.imread('circulo.png', cv2.IMREAD_GRAYSCALE)
ref_cross = cv2.imread('equis.png', cv2.IMREAD_GRAYSCALE)

# Función para ordenar nombres de archivos numéricamente
def numerical_sort(value):
    parts = value.split('.')
    try:
        return int(parts[0])
    except ValueError:
        return float('inf')

# Función para filtrar y ordenar las listas de imágenes
def get_sorted_image_list(folder):
    return sorted(
        [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.lower().endswith(('png', 'jpg', 'jpeg'))],
        key=numerical_sort
    )

# Obtener y ordenar las listas de imágenes
permitidos_images = get_sorted_image_list(permitidos_folder)
no_permitidos_images = get_sorted_image_list(no_permitidos_folder)

# Listas manuales para las posiciones y el jugador
movimientos_info = {
    'permitidos': {
        '1.png': (3, 3, 2, 2, 1),
        '2.png': (1, 1, 3, 3, 2),
        '3.png': (4, 4, 2, 2, 1),
        '4.png': (3, 1, 1, 1, 1),
        '5.png': (0, 0, 3, 2, 2),
        '6.png': (0, 1, 2, 1, 2),
        '7.png': (1, 3, 3, 3, 2),
        '8.png': (3, 3, 3, 1, 2),
        '9.png': (3, 1, 1, 3, 2),
        '10.png': (4, 0, 3, 1, 1),
        '11.png': (4, 1, 4, 0, 1),
        '12.png': (4, 2, 4, 1, 1),
        '13.png': (3, 1, 3, 2, 1),
        '14.png': (3, 2, 3, 3, 1),
        '15.png': (4, 3, 4, 3, 1),
        '16.png': (3, 3, 4, 4, 1),
        '17.png': (4, 4, 3, 3, 1),
        '18.png': (3, 3, 4, 4, 1),
        '19.png': (4, 4, 3, 3, 1),
        '20.png': (2, 3, 3, 3, 1),
        '21.png': (3, 0, 3, 1, 1),
        '22.png': (3, 1, 1, 1, 1),
        '23.png': (0, 1, 2, 1, 2),
        '24.png': (2, 3, 2, 2, 1),
        '25.png': (3, 2, 2, 2, 1),
        '27.png': (1, 2, 2, 2, 2),
        '28.png': (1, 1, 2, 2, 2),
        '29.png': (2, 2, 2, 0, 1),
        '30.png': (3, 1, 2, 2, 1),
        '31.png': (2, 2, 2, 0, 1),
        '33.png': (1, 1, 1, 2, 2),
        '34.png': (1, 3, 1, 2, 2),
        '35.png': (0, 2, 1, 3, 2),
        '36.png': (2, 4, 0, 2, 1),
        '37.png': (0, 2, 2, 2, 1),
        '38.png': (4, 2, 2, 4, 1),
        '39.png': (2, 0, 0, 2, 1),
        '40.png': (0, 0, 1, 1, 2),
        '41.png': (2, 2, 0, 0, 1),
        '42.png': (0, 1, 1, 1, 2),
        '43.png': (0, 2, 2, 0, 1),
        '44.png': (0, 3, 1, 3, 2),
        '45.png': (2, 3, 0, 3, 1),
        '46.png': (1, 4, 1, 3, 2),
        '47.png': (3, 4, 1, 4, 1),
        '48.png': (1, 4, 1, 2, 1),
        '49.png': (0, 0, 2, 0, 2),
        '50.png': (0, 2, 0, 0, 1),
        # La 26 y la 32 se borran por duplicadas
    },
    'no_permitidos': {
        '3.png': (2, 1, 2, 3, 2),
        '4.png': (1, 0, 4, 0, 2),
        '5.png': (4, 0, 4, 2, 2),
        '6.png': (1, 1, 3, 2, 2),
        '7.png': (2, 4, 4, 2, 1),
        '8.png': (4, 3, 4, 1, 1),
        '9.png': (3, 3, 1, 1, 1),
        '10.png': (3, 2, 4, 3, 2),
        '11.png': (1, 2, 4, 2, 2),
        '12.png': (0, 1, 3, 0, 2),
        '13.png': (2, 1, 4, 2, 2),
        '14.png': (4, 2, 1, 0, 1),
        '15.png': (3, 0, 2, 2, 1),
        '16.png': (1, 0, 4, 0, 1),
        '17.png': (0, 1, 1, 3, 1),
        '18.png': (0, 1, 3, 3, 2),
        '19.png': (1, 1, 3, 2, 2),
        '20.png': (4, 1, 1, 1, 2),
        '21.png': (4, 1, 2, 2, 2),
        '22.png': (2, 2, 0, 1, 1),
        '23.png': (1, 1, 3, 1, 2),
        '24.png': (4, 2, 3, 4, 1),
        '25.png': (4, 2, 3, 4, 1),
        '26.png': (1, 3, 4, 1, 2),
        '27.png': (4, 0, 2, 3, 1),
        '28.png': (4, 4, 4, 2, 2),
        '29.png': (2, 1, 4, 0, 1),
        '30.png': (4, 2, 1, 3, 2),
        '31.png': (4, 3, 2, 2, 1),
        '32.png': (0, 0, 2, 1, 2),
        '33.png': (4, 2, 4, 2, 1),
        '34.png': (2, 4, 4, 3, 2),
        '35.png': (1, 0, 4, 3, 1),
        '36.png': (4, 1, 2, 3, 2),
        '37.png': (2, 0, 4, 2, 1),
        '38.png': (2, 3, 0, 1, 2),
        '39.png': (4, 4, 2, 2, 1),
        '40.png': (2, 0, 0, 2, 2),
        '41.png': (4, 2, 2, 2, 1),
        '42.png': (1, 0, 1, 2, 2),
        '43.png': (4, 4, 0, 4, 1),
        '44.png': (2, 4, 0, 2, 2),
        '45.png': (3, 2, 2, 0, 1),
        '46.png': (4, 4, 3, 1, 2),
        '47.png': (1, 0, 4, 0, 1),
        '48.png': (3, 0, 0, 0, 2),
        '49.png': (3, 0, 1, 2, 1),
        '50.png': (0, 4, 1, 1, 2),
        #La 2 y la 3 se eliminan por duplicadas con otro grupo
    }
}

# Función para procesar una imagen
def process_image(image_path, previous_state, image_id, image_name, folder_type):
    I = Image.open(image_path)
    I1 = I.convert('L')
    I2 = np.asarray(I1, dtype=np.float64)
    X = I2.reshape((-1, 1))

    k_means = KMeans(n_clusters=3)
    k_means.fit(X)

    centroides = k_means.cluster_centers_
    etiquetas = k_means.labels_

    I2_compressed = np.choose(etiquetas, centroides.flatten())
    I2_compressed.shape = I2.shape
    I2_compressed = (I2_compressed - np.min(I2_compressed)) / (np.max(I2_compressed) - np.min(I2_compressed)) * 255
    I2_compressed = I2_compressed.astype(np.uint8)
    I2_image = Image.fromarray(I2_compressed)

    cell_size = 100
    cells = []
    for i in range(5):
        for j in range(5):
            cell = I2_image.crop((j * cell_size, i * cell_size, (j+1) * cell_size, (i+1) * cell_size))
            cells.append(cell)

    def identify_piece(cell):
        cell_array = np.asarray(cell, dtype=np.float64).reshape(-1, 1)
        k_means = KMeans(n_clusters=3)
        k_means.fit(cell_array)
        centroides = k_means.cluster_centers_
        etiquetas = k_means.labels_

        cell_cv = np.array(cell)
        if len(cell_cv.shape) == 2:
            cell_cv = cv2.cvtColor(cell_cv, cv2.COLOR_GRAY2BGR)

        sift = cv2.SIFT_create()
        kp_cell, des_cell = sift.detectAndCompute(cell_cv, None)

        if des_cell is None or len(kp_cell) == 0:
            return 0  # Celda vacía

        kp_circle, des_circle = sift.detectAndCompute(ref_circle, None)
        kp_cross, des_cross = sift.detectAndCompute(ref_cross, None)

        bf = cv2.BFMatcher()
        matches_circle = bf.knnMatch(des_cell, des_circle, k=2)
        matches_cross = bf.knnMatch(des_cell, des_cross, k=2)

        good_matches_circle = [m for m, n in matches_circle if m.distance < 0.90 * n.distance]
        good_matches_cross = [m for m, n in matches_cross if m.distance < 0.90 * n.distance]

        if len(good_matches_cross) > len(good_matches_circle):
            return 2  # Cruz
        else:
            return 1  # Círculo

    result_labels = []
    for cell in cells:
        label = identify_piece(cell)
        result_labels.append(label)

    column_names = [f'x{i}{j}' for i in range(5) for j in range(5)]
    df = pd.DataFrame([result_labels], columns=column_names)

    # Obtener la información manual para la imagen actual
    if folder_type == 'permitidos':
        info = movimientos_info['permitidos'].get(image_name, (0, 0, 0, 0, 0))
    else:
        info = movimientos_info['no_permitidos'].get(image_name, (0, 0, 0, 0, 0))

    pos_x_actual, pos_y_actual, pos_x_movimiento, pos_y_movimiento, jugador = info

    # Asignar validez basada en la carpeta
    validez = 1 if folder_type == 'permitidos' else 0

    df['POS_EN_X_ACTUAL'] = pos_x_actual
    df['POS_EN_Y_ACTUAL'] = pos_y_actual
    df['POS_EN_X_MOVIMIENTO'] = pos_x_movimiento
    df['POS_EN_Y_MOVIMIENTO'] = pos_y_movimiento
    df['JUGADOR'] = jugador
    df['VALIDEZ'] = validez
    df['IMAGE_ID'] = image_id
    #df['IMAGE_NAME'] = image_name

    return df, result_labels

results_df = pd.DataFrame()
previous_state = np.zeros((5, 5), dtype=int)

image_id = 0
for image_name in permitidos_images:
    image_path = os.path.join(permitidos_folder, image_name)
    df, new_state = process_image(image_path, previous_state=previous_state, image_id=image_id, image_name=image_name, folder_type='permitidos')
    results_df = pd.concat([results_df, df], ignore_index=True)
    image_id += 1

for image_name in no_permitidos_images:
    image_path = os.path.join(no_permitidos_folder, image_name)
    df, new_state = process_image(image_path, previous_state=previous_state, image_id=image_id, image_name=image_name, folder_type='no_permitidos')
    results_df = pd.concat([results_df, df], ignore_index=True)
    image_id += 1

# Ordenar el DataFrame por IMAGE_ID para mantener el orden original
results_df = results_df.sort_values(by='IMAGE_ID').drop(columns=['IMAGE_ID'])
results_df.to_csv('dataset.csv', index=False)
print(results_df)


Se han truncado las últimas 5000 líneas del flujo de salida.
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will 

    x00  x01  x02  x03  x04  x10  x11  x12  x13  x14  ...  x41  x42  x43  x44  \
0     2    2    2    2    2    2    2    2    2    2  ...    1    1    1    1   
1     2    2    2    2    2    2    0    2    2    2  ...    1    1    1    1   
2     2    2    2    2    2    2    0    2    2    2  ...    1    1    1    0   
3     2    2    2    2    2    2    1    2    2    2  ...    1    1    1    1   
4     0    2    2    2    2    2    0    2    2    2  ...    1    1    1    1   
..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
90    0    0    1    1    1    0    2    1    2    0  ...    1    0    0    0   
91    2    0    0    2    0    0    2    0    1    0  ...    0    1    0    1   
92    2    0    1    2    2    0    2    1    1    1  ...    0    0    0    0   
93    2    0    1    2    2    1    0    1    0    1  ...    0    0    0    0   
94    2    1    0    0    0    2    2    0    0    1  ...    1    2    1    0   

    POS_EN_X_ACTUAL  POS_EN

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in

**Código para eliminacion de duplicados con la dataset de cada grupo**

In [ ]:
import pandas as pd

# Ruta del archivo CSV original
input_file = 'Datasets_Grupos.csv'
# Ruta para guardar el nuevo archivo CSV sin duplicados
output_file = 'Dataset_sin_duplicadas.csv'

# Cargar el archivo CSV
df = pd.read_csv(input_file)

# Comprobar filas duplicadas
duplicated_rows = df[df.duplicated()]

# Imprimir las filas duplicadas (opcional)
print("Filas duplicadas encontradas:")
print(duplicated_rows)

# Eliminar filas duplicadas
df_cleaned = df.drop_duplicates()

# Guardar el DataFrame sin duplicados en un nuevo archivo CSV
df_cleaned.to_csv(output_file, index=False)

print(f"Archivo CSV sin duplicados guardado en: {output_file}")


Filas duplicadas encontradas:
    x00;x01;x02;x03;x04;x10;x11;x12;x13;x14;x20;x21;x22;x23;x24;x30;x31;x32;x33;x34;x40;x41;x42;x43;x44;POS_EN_X_ACTUAL;POS_EN_Y_ACTUAL;POS_EN_X_MOVIMIENTO;POS_EN_Y_MOVIMIENTO;JUGADOR;VALIDEZ
390  1;2;1;0;1;1;0;2;0;0;0;1;0;1;0;2;1;2;0;2;2;0;1;...                                                                                                                                         
484  1;2;1;0;1;1;1;2;0;0;0;1;2;1;0;0;0;2;0;2;2;0;1;...                                                                                                                                         
496  1;2;1;0;1;1;1;2;2;0;2;1;1;1;0;0;0;2;0;2;2;0;0;...                                                                                                                                         
500  1;2;1;0;1;1;1;2;0;0;0;1;2;1;0;0;0;2;0;2;2;0;1;...                                                                                                                                         
Archivo CS